# **Powered by 42RandomState**

#### **I created a secure access system using hand gesture recognition and facial detection with Python, OpenCV, and MediaPipe. The system starts by detecting my face to enable the password input phase. Then, I use my left hand to input a sequence of hand gestures (for example, 1, 2, 3, 4), which acts as my password. If I make a mistake, I can use my right hand to delete the last gesture. The system verifies the entered gesture sequence and either grants or denies access, showing a message and saving a screenshot accordingly. It also limits the number of login attempts for added security. Everything works in real-time through the webcam interface.**



## **ATTENTION!!** This code dosen't work on kaggle, you can download the notebook

In [1]:
#SETUP
!pip install mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 47.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.7 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.26.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0

In [2]:
password = input("Enter a 4-digit hand gesture password (digits 0–4): ")
while len(password) != 4 or not all(c in "01234" for c in password):
    password = input("Enter a 4-digit hand gesture password (digits 0–4): ")

gesture_password = list(password)

Enter a 4-digit hand gesture password (digits 0–4):  1234


In [ ]:
'''ACCESS WITH HAND PASSWORD'''
import cv2
import mediapipe as mp
import time
import os

window_name = "ACCESS WITH PASSWORD"
entered_gestures = []

mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
foto = 0

stable_threshold = 0.1 
gesture_stability_time = 1
last_gesture_time = 0
last_gesture_position = None

screenshot_dir = "screenshots"
os.makedirs(screenshot_dir, exist_ok=True)

def get_gesture(hand_landmarks):
    tips = [8, 12, 16, 20]
    open_fingers = 0
    for tip in tips:
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            open_fingers += 1
    if open_fingers == 0:
        return "0"
    elif open_fingers == 1:
        return "1"
    elif open_fingers == 2:
        return "2"
    elif open_fingers == 3:
        return "3"
    elif open_fingers == 4:
        return "4"
    return "unknown"

def get_hand_side(hand_landmarks, image_width):
    palm_center_x = hand_landmarks.landmark[9].x * image_width
    if palm_center_x < image_width / 2:
        return "Right"
    else:
        return "Left"

def calculate_distance(p1, p2):
    return ((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2) ** 0.5

cap = cv2.VideoCapture(0)
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 1000, 600)

volte = 0
time_between = 99999999999999999999999999999999999999999999999999999999999999

with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7) as hands, \
     mp_face.FaceDetection(min_detection_confidence=0.6) as face_detection:

    face_detected = False
    access_granted = False
    ready=True

    cancell = 0
    right_hand_present = False

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        face_results = face_detection.process(rgb)
        hand_results = hands.process(rgb)
        current_time = time.time()

        if volte < 3:
            if ready:
                gesture_label = "..."
                if hand_results.multi_hand_landmarks:
                    right_hand_present = True
                    for hand_landmarks in hand_results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        gesture = get_gesture(hand_landmarks)
                        gesture_label = gesture
                        hand_side = get_hand_side(hand_landmarks, frame.shape[1])

                        if hand_side == "Left":
                            palm_position = hand_landmarks.landmark[9]
                            cancell = 0
                            if last_gesture_position is not None:
                                distance = calculate_distance(palm_position, last_gesture_position)
        
                                if distance < stable_threshold:
                                    if current_time - last_gesture_time > gesture_stability_time:
                                        if len(entered_gestures) < 4:
                                            if len(entered_gestures) == 0 or entered_gestures[-1] != gesture:
                                                entered_gestures.append(gesture)
                                                last_gesture_time = current_time
                                else:
                                    last_gesture_time = current_time
                            last_gesture_position = palm_position
                            cancell = 0
                        
                        elif hand_side == "Right":
                            if not access_granted:
                                if len(entered_gestures) > 0 and len(entered_gestures) < 4:
                                    if cancell == 0:
                                        entered_gestures.pop()
                                elif len(entered_gestures) == 4:
                                    entered_gestures = []
                                    volte +=1
                                cancell += 1
                    
                else:
                    cancell = 0
                
                
                cv2.putText(frame, f"Available: 0, 1, 2, 3, 4", (50, 80),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                cv2.putText(frame, f"Attempts left: {3 - volte}", (400, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                cv2.putText(frame, f"act: {gesture_label}", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                cv2.putText(frame, f"Password: ", (50, 120), cv2.FONT_HERSHEY_SIMPLEX,
                                0.5, (200, 255, 100), 2)

                for i, g in enumerate(entered_gestures):
                    cv2.putText(frame, g, (150 + i*30, 120), cv2.FONT_HERSHEY_SIMPLEX,
                                0.5, (200, 255, 100), 2)

                if len(entered_gestures) == 4:
                    if entered_gestures == gesture_password:
                        access_granted = True
                        cv2.putText(frame, "ACCESS ALLOWED", (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                        if foto == 0:
                            filename = os.path.join(screenshot_dir, f"accesso_{int(current_time)}.png")
                            cv2.imwrite(filename, frame)
                            last_capture_time = current_time
                            foto += 1
                    else:
                        cv2.putText(frame, "ACCESS DENIED", (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                        if foto == 0:
                            filename = os.path.join(screenshot_dir, f"sbagliato_{int(current_time)}.png")
                            cv2.imwrite(filename, frame)
                            last_capture_time = current_time
                            foto += 1
        else:
            cv2.putText(frame, "ACCESS DENIED", (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            if foto == 1:
                filename = os.path.join(screenshot_dir, f"INTRUSO_{int(current_time)}.png")
                cv2.imwrite(filename, frame)
                last_capture_time = current_time
                foto += 1
            
        cv2.imshow(window_name, frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
        elif key == ord('r'):
            entered_gestures = []
            password_mode = face_detected = access_granted = False

cap.release()
cv2.destroyAllWindows()

2025-05-04 15:38:39.211246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746373119.401342      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746373119.451696      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[ WARN:0@12.978] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@12.979] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
